# Árboles de Decisión

Para esta guía veo que hay muchos ejercicios que se repiten, por ejemplo, armar árboles con ID3. No se realizarán todos pero se mostrará el algorítmo.

***2. Se quiere construir un árbol de decisión para clasificar si un conjunto de platos es adecuado para carnívoros (clase C) o veganos (clase V). El algoritmo CART debe analizar si hacer o no un split por la columna “ingrediente 2 en gramos” (columna numérica). Dados los valores de la columna y la clase a predecir, indicar cuál sería el valor de la ganancia de información para dicha columna:***

| Feature | Target |
|---------|--------|
| 23      | V      |
| 80      | C      |
| 85      | C      |
| 65      | V      |
| 100     | V      |
| 11      | C      |


Primero debemos ordenar los features de menor a mayor:

| Feature | Target |
|---------|--------|
| 11      | C      |
| 23      | V      |
| 65      | V      |
| 80      | C      |
| 85      | C      |
| 100     | V      |

Y podemos splittear de las diferentes formas:
```
C|VVCCV
CV|VCCV
CVV|CCV
CVVC|CV
CVVCC|V
```
Y para cada posible split calculamos su entropía.

La entropía de cada split es: 
```
C|VVCCV ->   H(i <= 11)/6 + 5*H(i > 11)/6 =   H(1/1, 0/1)/6 + 5*H(2/5, 3/5)/6 = 0.81 
CV|VCCV -> 2*H(i <= 23)/6 + 4*H(i > 23)/6 = 2*H(1/2, 1/2)/6 + 4*H(2/4, 2/4)/6 = 1
CVV|CCV -> 3*H(i <= 65)/6 + 3*H(i > 65)/6 = 3*H(1/3, 2/3)/6 + 3*H(2/3, 1/3)/6 = 0.92 
CVVC|CV -> 4*H(i <= 80)/6 + 2*H(i > 80)/6 = 4*H(2/4, 2/4)/6 + 2*H(1/2, 1/2)/6 = 1 
CVVCC|V -> 5*H(i <= 85)/6 +   H(i > 85)/6 = 5*H(3/5, 2/5)/6 +   H(0/1, 1/1)/6 = 0.81
```







Donde tenemos que tomar los splits que minimicen la entropía. Por lo tanto podemos cortar por el primer o último valor del dataframe, es decir, los que tienen entropía ~0.81.

***3. Dada la información sobre los siguientes días:***
$ $
$ $

| Día | Clima   | Temperatura | Humedad | Viento | ¿Jugar al tenis?  |
|-----|---------|-------------|---------|--------|-------------------|
| D1  | Soleado | Caluroso    | Alta    | Débil  | No                |
| D2  | Soleado | Caluroso    | Alta    | Fuerte | No                |
| D3  | Nublado | Caluroso    | Alta    | Débil  | Sí                |
| D4  | Lluvia  | Templado    | Alta    | Débil  | Sí                |
| D5  | Lluvia  | Frío        | Normal  | Débil  | Sí                |
| D6  | Lluvia  | Frío        | Normal  | Fuerte | No                |
| D7  | Nublado | Frío        | Normal  | Fuerte | Sí                |
| D8  | Soleado | Templado    | Alta    | Débil  | No                |
| D9  | Soleado | Frío        | Normal  | Débil  | Sí                |
| D10 | Lluvia  | Templado    | Normal  | Débil  | Sí                |
| D11 | Soleado | Templado    | Normal  | Fuerte | Sí                |
| D12 | Nublado | Templado    | Alta    | Fuerte | Sí                |
| D13 | Nublado | Caluroso    | Normal  | Débil  | Sí                |
| D14 | Lluvia  | Templado    | Alta    | Fuerte | No                |

***utilizar ID3 para determinar si las condiciones serán o no apropiadas para jugar al tenis.***


Como siempre arrancamos calculando la entropía del set de datos:

```
H(SET) = H(5/14,1 - 5/14) = 0.9402859586706311
```

In [11]:
from math import log2
entropy = lambda probas: -sum([p * log2(p) for p in probas])
entropy([5/14, 1 - 5/14])

0.9402859586706311

Y ahora debemos calcular la entropía de cada feature para saber donde maximizamos la ganancia de información.

```
H(viento)      = 8*H(6/8, 2/8)/14 + 6*H(1/2, 1/2)/14 = 0.8921
H(humedad)     = 7*H(3/7, 4/7)/14 + 7*H(6/7, 1/7)/14 = 0.7884
H(temperatura) = 4*H(2/4, 2/4)/14 + 6*H(4/6, 4/6)/14 + 4*H(3/4, 1/4)/14 = 0.8517
H(clima)       = 5*H(2/5, 3/5)/14 + 5*H(3/5, 2/5)/14 + 4*H(1, 0)/14     = 0.6935
```

Donde, por ejemplo, tenemos `8*H(Débil)/14 + 6*H(Fuerte)/14` para la entropía del viento.

Por lo tanto primero debemos tomar al clima y dentro del clima, como el estado nublado tiene la menor entropía, elegimos esta para que apunte directamente a `sí`. Ahora debemos ver que pasa cuando clima está soleado y con lluvia. 

### Lluvia

| Día | Clima   | Temperatura | Humedad | Viento | ¿Jugar al tenis?  |
|-----|---------|-------------|---------|--------|-------------------|
| D4  | Lluvia  | Templado    | Alta    | Débil  | Sí                |
| D5  | Lluvia  | Frío        | Normal  | Débil  | Sí                |
| D6  | Lluvia  | Frío        | Normal  | Fuerte | No                |
| D10 | Lluvia  | Templado    | Normal  | Débil  | Sí                |
| D14 | Lluvia  | Templado    | Alta    | Fuerte | No                |

Ahora debemos calcular las entropías dado que va a llover.

```
H(temperatura | llueve) = 3*H(2/3, 1/3)/5 + 2*H(1/2, 1/2)/5 = 0.9509
H(  humedad   | llueve) = 2*H(1/2, 1/2)/5 + 3*H(2/3, 1/3)/5 = 0.9509
H(   viento   | llueve) = 3*H(3/3, 0/3)/5 + 2*H(0/2, 2/2)/5 = 0
```

Y resulta que ahora debemos elegir, en el próximo nodo del árbol, en función del tipo de viento.

### Soleado

| Día | Clima   | Temperatura | Humedad | Viento | ¿Jugar al tenis?  |
|-----|---------|-------------|---------|--------|-------------------|
| D1  | Soleado | Caluroso    | Alta    | Débil  | No                |
| D2  | Soleado | Caluroso    | Alta    | Fuerte | No                |
| D8  | Soleado | Templado    | Alta    | Débil  | No                |
| D9  | Soleado | Frío        | Normal  | Débil  | Sí                |
| D11 | Soleado | Templado    | Normal  | Fuerte | Sí                |

Ahora debemos calcular las entropías dado que va a estar soleado.

```
H(temperatura | soleado) = 2*H(0/3, 3/3)/5 + 2*H(1/2, 1/2)/5 + H(1,0) = 0.4
H(  humedad   | soleado) = 2*H(2/2, 0/2)/5 + 3*H(0/3, 3/3)/5          = 0
H(   viento   | soleado) = 3*H(2/3, 1/3)/5 + 2*H(0/2, 2/2)/5          = 0.5509
```

Por lo que en este split decido en función de la humedad.

```
Clima Nublado -> Sí
Clima Soleado -> Humedad Alta   -> No
                 Humedad Normal -> Sí
Clima Lluvia  -> Viento Débil   -> Sí
                 Viento Fuerte  -> No
```

***5b. Explicar cómo hacer para evitar el overfitting y aplicarlo al árbol del punto anterior.***

Podemos usar min-bucket. Esta técnica tiene un hiperparámetro $k \in \mathbb{N}$, el cual corresponde a la cantidad mínima de registros que tiene que tener un feature para aparecer como nodo. Por ejemplo en el dataframe anterior, si tuvieramos que k = 6 entonces el clima soleado haría que se cancelen los partidos.

***7. Dados los siguientes puntos en una dimensión y sus labels para un problema de regresión (con el formato (coordenada_X, label)):***

```
(1, 6), (17, 11), (18, 14), (26, 23), (303, 108), (411, 109), (511, 211)
```

***Se quiere usar Gradient Boosting para aproximar los valores de forma tal de minimizar el error cuadrático medio (MSE por sus siglas en inglés).***

***a. Construir un árbol de decisión de nivel 1 y dibujarlo.***

***b. Construir un segundo árbol basado en Gradient Boosting para indicar la predicción final que se realizaría para cada punto y el error de entrenamiento en base a estas predicciones. Indicar si se cae en un caso de overfitting o underfitting.***